In [ ]:
import itertools
from functools import reduce
from itertools import product

import operator

K = Fields()

q = 2**61 - 1
Fq = GF(q)
x = polygen(Fq, 'x')
Fq2.<u> = Fq.extension(x^2+1)
log_n = 4
n = 2**log_n
v = vector(Fq, [Fq.random_element() for _ in range(n)])

def get_bit(n, k):
    return (n >> k) & 1

def int_to_bits(n, bit_length=None):
    if bit_length is None:
        bit_length = n.bit_length() if n != 0 else 1
    return [(n >> i) & 1 for i in range(bit_length)]

def multilinear_extension(v, F, var_names=None):
    n = len(v)
    m = n.bit_length() - 1
    if 2 ** m != n:
        raise ValueError("Vector length must be a power of 2.")
    
    if var_names is None:
        var_names = [f'X{i}' for i in range(1, m+1)]
    R = PolynomialRing(F, var_names)
    X = R.gens()
    p = R.zero()
    
    for i in range(n):
        term = F(v[i]) 
        bits = int_to_bits(i,m) 
        
        for j in range(m):
            if bits[j] == 1:
                term *= X[j]
            else:
                term *= (1 - X[j])
        p += term
    return p

v_tilde = multilinear_extension(v, Fq)

for i, t in enumerate(v):
  assert t == v_tilde([j for j in int_to_bits(i, log_n)])


def multilinear_matrix_extension(M, F, var_names=None):
    d, n = M.dimensions()
    total_elements = d * n
    
    m = total_elements.bit_length() - 1
    if 2^m != total_elements:
        raise ValueError("Matrix dimensions must multiply to a power of 2 (d * n = 2^m).")
    
    # Flatten the matrix into a vector (row-major order)
    v = M.list()
    
    if var_names is None:
        var_names = [f'X{i}' for i in range(1, m+1)]
    R = PolynomialRing(F, var_names)
    X = R.gens()
    
    p = R.zero()
    for i in range(total_elements):
        term = F(v[i])  
        bits = int_to_bits(i,m) 
        
        for j in range(m):
            if bits[j] == 1:
                term *= X[j]
            else:
                term *= (1 - X[j])
        
        p += term
    
    return p

M = Matrix(Fq, [[ Fq.random_element() for _ in range(n)] for _ in range(n)])
M_tilde = multilinear_matrix_extension(M, Fq)

for i, row in enumerate(M):
  for j, e in enumerate(row):
    assert M[i][j] == M_tilde([y for y in int_to_bits(j, log_n)]+[x for x in int_to_bits(i, log_n)])


# Build multi-linear basis polynomials
var_names = [f'X{i}' for i in range(log_n)]
R = PolynomialRing(Fq2, var_names)
X = R.gens()
print(X)
# print([X[i] for i in range(log_n)])
basis = []
for i in range(n):
    bits = [(i >> j) & 1 for j in range(log_n)]
    poly = 1
    for b, x in zip(bits, [X[i] for i in range(log_n)]):
        poly *= x if b else (1 - x)
    basis.append(poly)

# print(basis)

tilde_f_r = sum(v[i] * basis[i] for i in range(n))
v_tilde2 = multilinear_extension(v, Fq2)
for i, t in enumerate(v):
  params = [j for j in int_to_bits(i, log_n)]
  assert tilde_f_r(params) == v_tilde(params)


In [5]:
# 2.3 Rings and Modules
eta = 81
d = 54
q = 2**61 - 1
Fq = GF(q)
x = polygen(Fq, 'x')
Phi = x ** 54 + x ** 27 + 1
Rq.<u> = Fq.extension(Phi)
kappa = 16
## m = 2 ** 22 # TOO MUCH MEMORY
#m = 2 ** 8 # for testing purposes
#M = random_matrix(Rq, kappa, m)
a = Rq.random_element()

def cf(a):
    return a.list()

def cf_inv(a):
    return Rq(a)

def shift_matrix(Fq, Phi, d):
    F = matrix(Fq, d, d)
    c = cf(Phi)
    for i in range(d):
        for j in range(d):
            if i - 1 == j:
                F[i, j] = 1
            if j == d - 1:
                F[i, j] = -c[i]
    return F

def rot(a, Phi, Fq):
    cf_a = vector(Fq, a if isinstance(a, list) else cf(a))
    d = len(cf_a)
    F = shift_matrix(Fq, Phi, d)

    columns = [cf_a]
    F_power = F  # Start with F^1
    for _ in range(1, d):
        current = F_power * cf_a
        columns.append(current)
        F_power = F_power * F  # Compute next power incrementally
    rows = list(zip(*columns))
    M = matrix(Fq, len(cf_a), len(cf_a), sum([list(row) for row in rows], []))

    return M    # print([(F**i) * cf_a for i in range(1, d)])
    
# print(shift_matrix(Fq, Phi, d))
rot_a = rot(a, Phi, Fq)
b = Rq.random_element()
assert rot_a * vector(Fq, cf(b)) == vector(Fq,cf(a*b))

# F7 = GF(7)

# y = polygen(F7, 'y')
# Phi_test = y ** 3 + 1
# R7.<u> = F7.extension(Phi_test)

# a = 4*u**2 + 2*u + 3
# rot_a = rot(a, Phi_test, F7)
# print(rot_a)
# b = u**2 + 1
# print("cf(a*b)", cf(a*b))
# print("rot_a * vector(F7, cf(b)", rot_a * vector(F7, cf(b)))
# assert rot_a * vector(F7, cf(b)) == vector(F7, cf(a*b))

# 3.2 Neo’s solution, part-1: A matrix commitment scheme

def embed_Fq_to_Rq(z_i, d):
    """
    Embed a field element z_i ∈ Fq into Rq by using its bits
    as coefficients of a polynomial in Rq.
    """
    z_int = int(z_i)
    bits = [((z_int >> i) & 1) for i in range(d)]
    poly = sum(Fq(bits[i]) * u^i for i in range(len(bits)))
    return Rq(poly)

z_i = Fq.random_element()
z_i_embedded = embed_Fq_to_Rq(z_i, d)

print(f"Original Fq element: {z_i}")
print(f"Embedded Rq element: {z_i_embedded}")


Original Fq element: 2295263976831402827
Embedded Rq element: u^52 + u^51 + u^49 + u^46 + u^45 + u^43 + u^41 + u^38 + u^37 + u^35 + u^34 + u^32 + u^28 + u^26 + u^23 + u^22 + u^19 + u^16 + u^14 + u^9 + u^8 + u^6 + u^3 + u + 1


In [9]:
from random import randint


def decomp_b(z, d=None):
    F = z.base_ring()
    m = len(z)
    b = 2

    # Determine degree if not provided
    if d is None:
        # Find smallest d such that b^d > max(z)
        max_val = max([abs(int(zi)) for zi in z])
        d = 1
        while b**d <= max_val:
            d += 1

    # Initialize the result matrix
    # Z = matrix(F, d, m)

    # Perform b-ary decomposition
    cols = []
    for j in range(m):
        value = int(z[j])
        bits = [((value >> i) & 1) for i in range(d)]
        cols.append(bits)
    rows = list(zip(*cols))
    Z = matrix(F, d, m, sum([list(row) for row in rows], []))

    return Z


z_1 = Fq(123)
z_2 = Fq(567)
z_3 = Fq(890)
m = 3
z_1_embedded = embed_Fq_to_Rq(z_1, d)
z_2_embedded = embed_Fq_to_Rq(z_2, d)
z_3_embedded = embed_Fq_to_Rq(z_3, d)
z_vec = vector(Fq, [z_1, z_2, z_3])
print(z_vec)
Z = decomp_b(z_vec, d)
print("len(cf(z_1_embedded))", len(cf(z_1_embedded)))
print("len([Z[i, 0] for i in range(d)])", len([Z[i, 0] for i in range(d)]))
assert cf(z_1_embedded) == [Z[i, 0] for i in range(d)]
assert cf(z_2_embedded) == [Z[i, 1] for i in range(d)]
assert cf(z_3_embedded) == [Z[i, 2] for i in range(d)]

print("check sum", [Fq(sum([2**i * Z[i, col] for i in range(d)])) for col in range(m)])
assert list(z_vec) == [
    Fq(sum([2**i * Z[i, col] for i in range(d)])) for col in range(m)
]


def split_b(Z, k=None):
    F = Z.base_ring()
    d, m = Z.dimensions()
    b = 2

    # Determine depth if not provided
    if k is None:
        # Find smallest k such that b^k > max(Z)
        max_val = max([abs(int(Z[i, j])) for i in range(d) for j in range(m)])
        k = 1
        while b**k <= max_val:
            k += 1
    # Initialize the result matrices
    result = [matrix(F, d, m) for _ in range(k)]

    # Perform b-ary decomposition
    for i in range(d):
        for j in range(m):
            value = int(Z[i, j])
            bits = [((value >> i) & 1) for i in range(k)]
            for l in range(k):
                result[l][i, j] = F(bits[l])
    return result


max_val = 2**20  # low norm
test = matrix(Fq, d, m)
for i in range(d):
    for j in range(m):
        test[i, j] = Fq(randint(0, max_val - 1))
Z = split_b(test)
Z_restored = matrix(Fq, d, m)
for i, z in enumerate(Z):
    Z_restored += (2**i) * z

print(Z_restored)
assert test == Z_restored

(123, 567, 890)
len(cf(z_1_embedded)) 54
len([Z[i, 0] for i in range(d)]) 54
check sum [123, 567, 890]
[ 787410  592271   68327]
[ 982146   98733  650885]
[ 927832 1008950  121073]
[ 819977  633794  197135]
[  40074  560469   86757]
[ 617157  207479  957146]
[ 779954  916961  996396]
[ 776539  892294  643396]
[ 756744  495502 1046642]
[ 206254  237853  198113]
[ 269375  444557  587954]
[ 712083 1037619  603519]
[ 686168  845660  665960]
[ 231446  378886  738925]
[  58776  279412  991761]
[ 886468  397154  231586]
[ 661197  923410   75331]
[ 318005  628255  394598]
[ 561215  175287  515037]
[ 327748  274943  796167]
[  25519  671310  888381]
[ 875969  899308  946189]
[ 865400  689186  704203]
[  21870  383690   71762]
[ 466969  290416  470402]
[ 140923   77975 1014223]
[ 799887  283203  993608]
[ 269002  144439  817993]
[ 471000  315295  384526]
[ 213056  188191   36515]
[ 404453  942563  973875]
[ 324884  995065  312255]
[ 704729  467305  515688]
[ 626266   69907  225681]
[ 131858  425

In [ ]:
import json

with open("../mimc_r1cs.json", "r") as f:
    r1cs = json.load(f)

z = vector(Fq, [int(w, base=16) for w in r1cs["witness"]])
Z = decomp_b(z)
assert d == Z.dimensions()[0]
m = Z.dimensions()[1]
print("Z.dimensions()", Z.dimensions())
for i in range(Z.nrows()):
    for j in range(Z.ncols()):
        assert Z[i, j] != 0 or Z[i, j] != 1

# In particular, each column of a low-norm matrix gets mapped as the coefficients of a single ring element.
z_prime = vector(Rq, [cf_inv(list(Z.column(j))) for j in range(Z.ncols())])

pp = random_matrix(Rq, kappa, m)
print("pp.dimensions()", pp.dimensions())
print("type(pp)", type(pp))
print("z_prime", type(z_prime))
c = pp * z_prime

rows = []
for x in c:
    rows.append(cf(x))
C = matrix(Fq, kappa, d, sum([list(row) for row in rows], [])).transpose()
print("C.dimensions()", C.dimensions())

Z.dimensions() (54, 71)
pp.dimensions() (16, 71)
type(pp) <class 'sage.matrix.matrix_generic_dense.Matrix_generic_dense'>
z_prime <class 'sage.modules.free_module_element.FreeModuleElement_generic_dense'>
C.dimensions() (54, 16)


In [ ]:
print("check sum", [sum(b**row * [Z[row, i] for i in range(d)]) for row in range(3)])